In [1]:
import wandb

In [ ]:
api_key= 'df0e0561866b4c4a20c92ff2adea2a2a789e0199'
wandb.login(key=api_key)

SINGLE TRIAL

In [ ]:
# Function to parse a text file with key-value pairs (e.g., hyperparameters or options)
def parse_text_file(file_path, separator=':'):
    config_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith("------------"):  # Skip empty lines and separators
                key, value = line.split(separator)
                key, value = key.strip(), value.strip()
                # Convert value to float, int, or keep as string if needed
                try:
                    value = float(value) if '.' in value else int(value)
                except ValueError:
                    pass  # Keep as string if it cannot be converted
                config_dict[key] = value
    return config_dict

""" # Parse the hyperparameters (from hyperparameters.txt)
hyperparameters = parse_text_file('Optuna/Cycle_2/trial_0/hyperparameters.txt') """

# Parse the options/configuration (from opt.txt)
configuration = parse_text_file('Optuna/Cycle_2/trial_9/opt.txt')

""" # Combine both dictionaries into one
config = {**hyperparameters, **configuration} """

# Initialize your WandB run
wandb.init(project="my_project", name="trial_9", config=configuration)

# Open the log_train.txt file and read it
with open("Optuna/Cycle_2/trial_9/log_train.txt", "r") as file:
    lines = file.readlines()

# Regular expressions for extracting the numerical data
metrics_regex = {
    "train_loss": r"Train_loss\s*:\s*([\d\.]+)",
    "valid_loss": r"Valid_loss\s*:\s*([\d\.]+)",
    "current_accuracy": r"Current_accuracy\s*:\s*([\d\.]+)",
    "learning_rate": r"Learning_rate\s*:\s*([\d\.e\-]+)",
    "batch_cer": r"Batch_CER\s*:\s*([\d\.]+)",
    "best_accuracy": r"Best_accuracy\s*:\s*([\d\.]+)",
    "best_norm_ed": r"Best_norm_ED\s*:\s*([\d\.]+)",
    "average_cer": r"Average_CER\s*:\s*([\d\.]+)"
}

# Regular expression for extracting Ground Truth | Prediction | Confidence | CER lines
prediction_regex = r"([^\|]+)\s*\|\s*([^\|]+)\s*\|\s*([\d\.]+)\s*\|\s*([\w]+)\s*\|\s*([\d\.]+)"

# Variables to store extracted metrics
metrics = {
    "train_loss": None,
    "valid_loss": None,
    "current_accuracy": None,
    "learning_rate": None,
    "batch_cer": None,
    "best_accuracy": None,
    "best_norm_ed": None,
    "average_cer": None,
}

# List to store predictions, ground truths, and CERs for logging
predictions = []

# Loop through each line to find and log metrics
for line in lines:
    # Check if the line contains any of the numerical metrics
    for metric, pattern in metrics_regex.items():
        match = re.search(pattern, line)
        if match:
            metrics[metric] = float(match.group(1))
    
    # Check if the line matches the prediction structure
    match = re.search(prediction_regex, line)
    if match:
        ground_truth = match.group(1)
        prediction = match.group(2)
        confidence = float(match.group(3))
        true_false = match.group(4)
        cer = float(match.group(5))
        
        # Append the prediction details for later logging
        predictions.append({
            "Ground Truth": ground_truth,
            "Prediction": prediction,
            "Confidence": confidence,
            "True/False": true_false,
            "Character Error Rate (CER)": cer
        })
    
    # If there's a line separator (e.g., dashes or similar), log metrics if available
    if line.startswith("-"):
        # Log the numerical metrics to WandB if they were found in the previous lines
        if metrics["train_loss"] is not None:
            wandb.log({
                "train_loss": metrics["train_loss"],
                "valid_loss": metrics["valid_loss"],
                "current_accuracy": metrics["current_accuracy"],
                "learning_rate": metrics["learning_rate"],
                "batch_cer": metrics["batch_cer"],
                "best_accuracy": metrics["best_accuracy"],
                "best_norm_ed": metrics["best_norm_ed"],
                "average_cer": metrics["average_cer"]
            })
            
            # Reset metrics after logging
            metrics = {key: None for key in metrics}

# Log predictions and CER (you can log them as a table or as individual values)
for prediction in predictions:
    wandb.log({
        "Ground Truth": prediction["Ground Truth"],
        "Prediction": prediction["Prediction"],
        "Confidence": prediction["Confidence"],
        "True/False": prediction["True/False"],
        "Character Error Rate (CER)": prediction["Character Error Rate (CER)"]
    })

# Finish the run
wandb.finish()

MULTIPLE TRIALS FROM A FOLDER

In [5]:
import os
import re
import wandb

# Function to parse a text file with key-value pairs (e.g., hyperparameters or options)
def parse_text_file(file_path, separator=':'):
    config_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith("------------"):  # Skip empty lines and separators
                if separator in line:
                    key, value = line.split(separator, 1)  # Only split at the first occurrence
                    key, value = key.strip(), value.strip()
                    # Convert value to float, int, or keep as string if needed
                    try:
                        value = float(value) if '.' in value else int(value)
                    except ValueError:
                        pass  # Keep as string if it cannot be converted
                    config_dict[key] = value
    return config_dict


# Regular expressions for extracting numerical data
metrics_regex = {
    "train_loss": r"Train loss:\s*([\d\.]+)",
    "valid_loss": r"Valid loss:\s*([\d\.]+)",
    "current_accuracy": r"Current_accuracy\s*:\s*([\d\.]+)",
    "best_accuracy": r"Best_accuracy\s*:\s*([\d\.]+)",
    "best_norm_ed": r"Best_norm_ED\s*:\s*([\d\.]+)",
}

# Regular expression for extracting Ground Truth | Prediction | Confidence | CER lines
prediction_regex = r"([^\|]+)\s*\|\s*([^\|]*)\s*\|\s*([\d\.]+)\s*([\w]+)"

# Main function to process all trials in a folder
def log_trials_from_folder(base_folder, project_name):
    for trial_folder in os.listdir(base_folder):
        trial_path = os.path.join(base_folder, trial_folder)
        if os.path.isdir(trial_path):
            # Parse configuration files
            hyperparameters_path = os.path.join(trial_path, 'hyperparameters.txt')
            opt_path = os.path.join(trial_path, 'opt.txt')
            log_path = os.path.join(trial_path, 'log_train.txt')

            hyperparameters = parse_text_file(hyperparameters_path) if os.path.exists(hyperparameters_path) else {}
            configuration = parse_text_file(opt_path) if os.path.exists(opt_path) else {}

            # Combine both dictionaries
            config = {**hyperparameters, **configuration}

            # Initialize WandB run
            wandb.init(project=project_name, name=trial_folder, config=config)

            # Read and process the log_train.txt file
            if os.path.exists(log_path):
                with open(log_path, "r") as file:
                    lines = file.readlines()

                metrics = {}

                for line in lines:
                    # Extract metrics dynamically
                    for metric, pattern in metrics_regex.items():
                        match = re.search(pattern, line)
                        if match:
                            metrics[metric] = float(match.group(1))

                    # Log metrics only when a separator line is encountered
                    if line.startswith("-") and metrics:
                        wandb.log(metrics)
                        metrics = {}

                    # Extract predictions
                    match = re.search(prediction_regex, line)
                    if match:
                        ground_truth = match.group(1).strip()
                        prediction = match.group(2).strip() or "N/A"
                        try:
                            confidence = float(match.group(3))
                            true_false = match.group(4)
                            wandb.log({
                                "Ground Truth": ground_truth,
                                "Prediction": prediction,
                                "Confidence": confidence,
                                "True/False": true_false,
                            })
                        except ValueError:
                            continue  # Skip invalid prediction lines

            # Finish the WandB run
            wandb.finish()

In [10]:
# Specify the base folder containing trials and the project name
base_folder = "/home/af-ml-dev/JFreaks/OCR/EasyOCR/deep-aug/deep-text-recognition-benchmark/optuna/vitstr_study_4"
project_name = "vitstr_study_4"

# Call the function
log_trials_from_folder(base_folder, project_name)

Confidence,▁▁▁▁▁▁▂▁▁▃▂▂▅▅▂█▄▆█▄▄▅▂▂▂▅▆▄█▃▂▃▆▄▁▂▆█▃▅
best_accuracy,▁▁▁▁▁▁▁▁▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇████████████████
best_norm_ed,▁▁▁▁▂▂▃▅▇▇▇▇▇▇██████████████████████████
current_accuracy,▁▁▁▁▁▁▁▁▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇███████████████
train_loss,█▇▆▆▆▆▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▇▆▆▆▆▅▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Confidence,0.4892
Ground Truth,jv9g7
Prediction,jv9g7
True/False,True
best_accuracy,78.917


Confidence,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▂▁▃▆▂▂▅▃▂▁█▂▄▇▄▄▄▁▃
best_accuracy,▁▁▁▁▁▁▁▁▁▂▂▅▆▆▆▇▇████████
best_norm_ed,▁▁▁▁▂▂▂▄▅▆▆▇▇▇███████████
current_accuracy,▁▁▁▁▁▁▁▁▁▂▂▅▆▆▆▇▇████████
train_loss,█▇▆▆▆▆▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▇▆▆▆▆▆▄▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Confidence,0.1705
Ground Truth,hdii9
Prediction,hdii9
True/False,True
best_accuracy,64.583


Confidence,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▂▂▁▁▅▃▃▂█▄▅▅▄▆▆▂▇▅▇▄█▃
best_accuracy,▁▁▁▁▁▁▁▂▃▄▄▆▇▇▇▇█████████
best_norm_ed,▁▁▁▁▂▂▃▅▆▇▇██████████████
current_accuracy,▁▁▁▁▁▁▁▂▃▄▄▆▇▇▇▇█████████
train_loss,█▇▆▆▆▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▇▆▆▆▆▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Confidence,0.4003
Ground Truth,v5wyek
Prediction,v5wyek
True/False,True
best_accuracy,70.967


Confidence,▁▁▁▁▁▁▁▁▁▁▁▂▂▆▂▁▃▃▇▅▂▇▂█▂▃▃▂▃▂▄▅▃▇▅▁▂▂▆▄
best_accuracy,▁▁▁▁▁▁▁▂▅▅▆▆▆▆▇▇▇▇▇▇█████████████████
best_norm_ed,▁▁▁▂▂▃▅▆▇▇▇▇█████████████████████████
current_accuracy,▁▁▁▁▁▁▁▂▅▅▆▆▆▆▇▇▇▇▇▇█████████████████
train_loss,█▇▆▆▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▇▆▆▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Confidence,0.466
Ground Truth,xwsn4
Prediction,xwsn4
True/False,True
best_accuracy,76.933


Confidence,▁▁▁▁▁▁▁▁▁▁█▅▃▄▆▂▂▃▃▆▄▄▃▂▅▄▁▃▃▆▄▄█▄▃█▄▆▂▆
best_accuracy,▁▁▁▁▁▁▁▃▅▆▆▇▇▇▇▇▇▇▇█████████████████████
best_norm_ed,▁▁▁▂▂▄▅▆▇▇██████████████████████████████
current_accuracy,▁▁▁▁▁▁▁▃▅▆▆▇▇▇▇▇▇▇▇█████████████████████
train_loss,█▇▆▆▆▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▇▆▆▆▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Confidence,0.4447
Ground Truth,5rmr3v
Prediction,5rmr3v
True/False,True
best_accuracy,80.483
